In [1]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm


MODEL_NAME = "Helsinki-NLP/opus-mt-en-vi"

tokenizer = MarianTokenizer.from_pretrained(MODEL_NAME)
model = MarianMTModel.from_pretrained(MODEL_NAME)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-vi. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

c:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

In [3]:
INPUT_FILE = "../data/train/train.csv"     # đổi thành train / val / test
OUTPUT_FILE = "../data/train/train_vi.csv"

df = pd.read_csv(INPUT_FILE)

texts = df["text"].tolist()


def translate_batch(texts, batch_size=16):
    translated_texts = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        encoded = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        with torch.no_grad():
            generated = model.generate(**encoded)

        translated = tokenizer.batch_decode(
            generated,
            skip_special_tokens=True
        )
        translated_texts.extend(translated)
    return translated_texts


df["text_vi"] = translate_batch(texts)


df_vi = df[["text_vi", "emotion", "risk"]]
df_vi = df_vi.rename(columns={"text_vi": "text"})

df_vi.to_csv(OUTPUT_FILE, index=False)

print("✅ Translation completed:", OUTPUT_FILE)


  0%|          | 9/9242 [11:46<201:27:58, 78.55s/it]


KeyboardInterrupt: 